# Task 3: Modeling Optimization

**Course:** Introduction to Data Science
**Lecturer:** Prof. Dr. Hendrik Meth

**Group 2:**
- Linus Breitenberger
- Tristan Ruhm
- Prarichut Poachanuan
- Anushka Irphale
- Patryk Gadziosmki

<div style="width:100%;height:30px;background-color:#E31134"></div>

<div style="width:100%;height:30px;background-color:#E31134"></div>

## 6. Summary of Task 3

# 1. Reworking and Optimization of train and test data

Since our final model for Task 2 didn't seem to be optimal and some details had been overseen, we decided to do a rework of our train and test datas and optimize it.

Because of the high correlation with 'cnt', we decided to keep 'instant' in the data set. The features 'temp' and 'atemp' had a multicorrelation, so we kept 'atemp', which had a minimal better correlation, in and dropped 'temp'. We also removed the 'casual' and 'registered' labels, since we only use 'cnt'.

In the end we tested our models with the new test data and had following result:
- MAE: 923.977 
- R^2 value of the model:  0.36870153666640637

The Mean Absolute Error has an improvement. On the opposite, the determination R^2 has a deterioration. We exported the reworked train- and testdata to csv and used for the model optimization.


# 2. Model Optimization

## 2.1 Split

In our first step, we refined the feature and label for our model buidling with splitting the training data into `train_features` and `train_labels` and the testdata into `test_features` and `test_label`.

## 2.2 Model Building

For a better model optimization, we created further model versions. Beside of linear regression, scikit-learn has various other algorithms. With testing these algortihms, we can figure out, which model has the best result. In the following, we present the results of each algorithm.

For each model, we used the method forward selection, to get the best result.

### 2.2.1 Polynomial Regression

First, we initiated a model with the algorithm polynomial regression.

For that, we used the function `makepipeline(PolynomialFeatures())`. It gives out multiple results, which  will run through `linear_model.LinearRegression()`, to get one value as a prediction.

### 2.2.2 K-Nearest-neighbours Regression

With the K-nearest-neighbours regression, we compared each value of 'cnt' with his 10 nearest neighbour-values. For that, we used the function `KNeighborsRegressor()` to predict a numeric label, which is the average of all 10 values. A special aspect is that no linear relationships are required to build a model.

### 2.2.3 Regression Tree / Decision Tree Regression

A regression tree makes it possible to handle nominal features and non-linear relationships. We used the `DecisionTreeRegressor()`-function.

### 2.2.4 Support Vector Regression

The Support Vector Regression defines a hyperplane, which can be linear, polynomial or gaussian.

First, we used the function `SVR()` with a polynomial kernel, since the evaluation of polynomial regression had a good result. The result of it was not so good, so we concluded to take the linear kernel, which had a better result.

## 2.3 Evaluation

## 2.4 Interpretation & Conlcusion